In [1]:
import os
import tqdm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

In [2]:
path = './data/ml-latest-small/'

ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [3]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
movies_df.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


# Representation using tf-idf

## Genre Representation

In [6]:
unique_genres = list(set([gr for grs in list(map(lambda x: x.split('|'), movies_df['genres'])) for gr in grs]))
total_genres = len(unique_genres)
total_movies = len(movies_df)

print(f'num of moives: {total_movies}\nnum of genres: {total_genres}')

num of moives: 9742
num of genres: 20


In [7]:
genre_count = dict.fromkeys(unique_genres)

for genre_list in movies_df['genres']:
    for genre in genre_list.split('|'):
        if genre_count[genre] == None:
            genre_count[genre] = 1
        else:
            genre_count[genre] += 1
            
print(genre_count)

{'Western': 167, 'Horror': 978, 'Thriller': 1894, 'Comedy': 3756, 'Film-Noir': 87, 'Documentary': 440, 'Sci-Fi': 980, 'Animation': 611, 'IMAX': 158, 'Crime': 1199, 'Musical': 334, 'Drama': 4361, 'Romance': 1596, 'Fantasy': 779, 'Children': 664, 'War': 382, 'Mystery': 573, 'Action': 1828, 'Adventure': 1263, '(no genres listed)': 34}


In [8]:
'''
Only idf is used because the genre appears once for each movie.
'''
for genre in genre_count:
    genre_count[genre] = np.log10(total_movies/genre_count[genre])
    
print(genre_count)

{'Western': 1.7659316540881678, 'Horror': 0.9983092704481497, 'Thriller': 0.7112681505684965, 'Comedy': 0.4139225416416778, 'Film-Noir': 2.0491288726171324, 'Documentary': 1.3451954487495636, 'Sci-Fi': 0.9974220495432563, 'Animation': 1.2026069149931968, 'IMAX': 1.7899910382813284, 'Crime': 0.9098289421369025, 'Musical': 1.4649016584241867, 'Drama': 0.3490620385623247, 'Romance': 0.7856152382210405, 'Fantasy': 1.0971106675631865, 'Children': 1.1664800458677336, 'War': 1.4065847623240424, 'Mystery': 1.2304935032683613, 'Action': 0.7266719338379385, 'Adventure': 0.8872447746804204, '(no genres listed)': 2.457169208193496}


In [9]:
%%time
'''
Create Genre Representation
It takes about 40 seconds
'''
genre_representation = pd.DataFrame(columns=sorted(unique_genres), index=movies_df.index)

for index, row in movies_df.iterrows():
    dict_temp = {g: genre_count[g] for g in row['genres'].split('|')}
    row_to_add = pd.DataFrame(dict_temp, index=[index])
    genre_representation.update(row_to_add)

genre_representation.head()

Wall time: 52.4 s


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,0.887245,1.202607,1.16648,0.413923,NaN,NaN,NaN,1.097111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0.887245,NaN,1.16648,NaN,NaN,NaN,NaN,1.097111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,0.413923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.785615,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.413923,NaN,NaN,0.349062,NaN,NaN,NaN,NaN,NaN,NaN,0.785615,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,0.413923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Tag Representation
- TfidfVectorizer (wrong case)
- Module not used (only use python)

In [10]:
tag_list = list(map(lambda x: x.split(','), tags_df['tag']))
unique_tags = list(set(list(map(lambda x: x.strip(), list([t for tags in tag_list for t in tags])))))
total_tags = len(unique_tags)

print(f'num of tags:{len(tag_list)} \nnum of unique tags: {total_tags}')

num of tags:3683 
num of unique tags: 1589


In [11]:
start_id = 60756
corpus = ['']
cnt = 0
for _, rows in tags_df[['movieId','tag']].iterrows():
    m_id, tag = rows
    if start_id == m_id:
        corpus[cnt] = corpus[cnt] + ' ' + tag
    else:
        corpus.append('')
        start_id = m_id
        cnt+=1
        corpus[cnt] = corpus[cnt] + ' ' + tag
        
#print(corpus)

[' funny Highly quotable will ferrell', ' Boxing story MMA Tom Hardy', ' drugs Leonardo DiCaprio Martin Scorsese', ' way too long', ' Al Pacino gangster mafia', ' Al Pacino Mafia', ' holocaust true story', ' twist ending', ' Anthony Hopkins courtroom drama twist ending', ' britpop indie record label music', ' dumpster diving Sustainability', ' romantic comedy wedding', ' painter', ' bloody', ' black hole sci-fi time-travel', ' fantasy magic board game Robin Williams', ' beautiful scenery epic historical inspirational Medieval mel gibson Oscar (Best Cinematography) revenge sword fight', ' black comedy Christina Ricci Christopher Lloyd dark comedy family gothic', ' Al Pacino Andy Garcia Classic Francis Ford Coppola mafia', ' black comedy Christina Ricci Christopher Lloyd Family gothic quirky', ' family funny Macaulay Culkin sequel', ' animation Disney funny original Pixar sequel Tom Hanks', ' ancient Rome Epic history imdb top 250 revenge Rome Russell Crowe', ' Ed Harris Jude Law Rachel 

In [12]:
'''
using TfidfVectorizer
'''

from sklearn.feature_extraction.text import TfidfVectorizer

tag_representation = TfidfVectorizer().fit_transform(corpus)

In [13]:
tag_representation

<1771x1744 sparse matrix of type '<class 'numpy.float64'>'
	with 5434 stored elements in Compressed Sparse Row format>

<b>tags are not in word units<b> <br>
TfidfVectorizer splits tags made up of sentences and uses them.<br> 
Therefore, it does not match the existing number of unique tags.<br>
    
- origin num of unique tags : 1589
- TfidfVectorizer tags : 1744

In [14]:
'''
Compute Idf for tag (Module not used)
'''
movie_tag_count = len(set(tags_df['movieId']))
tag_count_dict = dict.fromkeys(unique_tags)

for movie_tag_list in tags_df['tag']:
    for tag in movie_tag_list.split(','):
        if tag_count_dict[tag.strip()] == None:
            tag_count_dict[tag.strip()] = 1
        else:
            tag_count_dict[tag.strip()] += 1
            
tag_idf = dict()
for tag in tag_count_dict:
    tag_idf[tag] = np.log10(movie_tag_count / tag_count_dict[tag])

#print(tag_idf)

{'Casey Affleck': 3.196452541703389, 'Morrow': 3.196452541703389, 'Liam Neeson': 2.7193312869837265, 'conspiracy': 3.196452541703389, 'MacBeth': 3.196452541703389, 'Francis Ford Coppola': 3.196452541703389, 'Lou Gehrig': 3.196452541703389, 'Amish': 2.895422546039408, 'Doc Ock': 3.196452541703389, 'lack of story': 3.196452541703389, 'nightmare': 3.196452541703389, 'cheeky': 3.196452541703389, 'big boys with guns': 3.196452541703389, 'revolution': 3.196452541703389, 'carnival': 3.196452541703389, 'rape': 2.895422546039408, 'dating': 2.895422546039408, 'tension building': 3.196452541703389, 'Agatha Christie': 2.895422546039408, 'real estate': 3.196452541703389, 'surreal': 1.8347247056857963, 'bad ass': 3.196452541703389, 'writing': 2.7193312869837265, 'president': 2.5943925503754266, 'hope': 3.196452541703389, 'biking': 3.196452541703389, 'Jean Reno': 2.895422546039408, 'remake': 2.196452541703389, 'intelligent': 2.351354501689132, 'Brad Pitt': 2.4974825373673704, 'Studio Ghibli': 2.89542

In [15]:
%%time
'''
Create Tag Representation
It takes about 4 minutes
'''
tag_representation = pd.DataFrame(columns=sorted(unique_tags), index=list(set(tags_df['movieId'])))
for name, group in tags_df.groupby(by='movieId'):
    temp_list = list(map(lambda x: x.split(','), list(group['tag'])))
    temp_tag_list = list(set(list(map(lambda x: x.strip(), list([tag for tag_list in temp_list for tag in tag_list])))))
    
    dict_temp = {i: tag_idf[i.strip()] for i in temp_tag_list}
    row_to_add = pd.DataFrame(dict_temp, index=[group['movieId'].values[0]])
    tag_representation.update(row_to_add)
    
tag_representation = tag_representation.sort_index(0)

Wall time: 4min 7s


In [16]:
tag_representation

,"""artsy""",06 Oscar Nominated Best Movie - Animation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001-like,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
184471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Concat the Representation

In [17]:
movie_representation = pd.concat([genre_representation, tag_representation], axis=1).fillna(0)

In [18]:
movie_representation

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
1,0.0,0.000000,0.887245,1.202607,1.16648,0.413923,0.0,0.0,0.000000,1.097111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.887245,0.000000,1.16648,0.000000,0.0,0.0,0.000000,1.097111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.00000,0.413923,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.000000,0.000000,0.00000,0.413923,0.0,0.0,0.349062,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.000000,0.000000,0.00000,0.413923,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.726672,0.000000,1.202607,0.00000,0.413923,0.0,0.0,0.000000,1.097111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.000000,0.000000,1.202607,0.00000,0.413923,0.0,0.0,0.000000,1.097111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.349062,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193587,0.0,0.726672,0.000000,1.202607,0.00000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
%%time
def custom_cosine_similarity(x, mode='similarity'):
    if mode == 'similarity':
        return np.dot(x, x.T) / (np.linalg.norm(x, axis=1)[:, np.newaxis] * np.linalg.norm(x, axis=1)[np.newaxis, :])
    if mode == 'distance':
        return 1 - np.dot(x, x.T) / (np.linalg.norm(x, axis=1)[:, np.newaxis] * np.linalg.norm(x, axis=1)[np.newaxis, :])
    
result  = custom_cosine_similarity(movie_representation)
result_df = pd.DataFrame(data=result, index=[movie_representation.index])

Wall time: 2.2 s


In [20]:
result_df

,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
1,1.000000,0.124438,0.008403,0.040571,0.011755,0.000000,0.016339,0.331122,0.000000,0.131794,...,0.064466,0.260941,0.071492,0.271710,0.0,0.348295,0.379492,0.000000,0.232553,0.093519
2,0.124438,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240843,0.000000,0.095861,...,0.000000,0.000000,0.000000,0.000000,0.0,0.108082,0.117763,0.000000,0.000000,0.000000
3,0.008403,0.000000,1.000000,0.179391,0.011294,0.000000,0.072246,0.000000,0.000000,0.000000,...,0.006560,0.000000,0.068686,0.000000,0.0,0.020322,0.022142,0.000000,0.000000,0.089849
4,0.040571,0.000000,0.179391,1.000000,0.054530,0.000000,0.348828,0.000000,0.000000,0.000000,...,0.031674,0.101979,0.567487,0.000000,0.0,0.098119,0.106908,0.365843,0.000000,0.433821
5,0.011755,0.000000,0.011294,0.054530,1.000000,0.000000,0.640342,0.000000,0.000000,0.000000,...,0.009177,0.000000,0.096091,0.000000,0.0,0.028429,0.030976,0.000000,0.000000,0.125697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.348295,0.108082,0.020322,0.098119,0.028429,0.211466,0.039515,0.000000,0.397065,0.213809,...,0.206804,0.631077,0.172901,0.657123,0.0,1.000000,0.917791,0.000000,0.767770,0.226174
193583,0.379492,0.117763,0.022142,0.106908,0.030976,0.000000,0.043055,0.000000,0.000000,0.000000,...,0.169874,0.687605,0.188388,0.715984,0.0,0.917791,1.000000,0.000000,0.612800,0.246433
193585,0.000000,0.000000,0.000000,0.365843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.278750,0.644671,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000
193587,0.232553,0.000000,0.000000,0.000000,0.000000,0.275428,0.000000,0.000000,0.517166,0.278480,...,0.247849,0.821961,0.000000,0.855885,0.0,0.767770,0.612800,0.000000,1.000000,0.000000


In [21]:
from sklearn.metrics.pairwise import cosine_similarity
'''
Same result as custom function
'''


def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    cs_df = pd.DataFrame(data=cos_sim, index=[a.index])

    return cs_df


cs_df = cos_sim_matrix(movie_representation, movie_representation)
cs_df

,0,1,2,3,4,5,6,7,8,9,...,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
1,1.000000,0.124438,0.008403,0.040571,0.011755,0.000000,0.016339,0.331122,0.000000,0.131794,...,0.064466,0.260941,0.071492,0.271710,0.0,0.348295,0.379492,0.000000,0.232553,0.093519
2,0.124438,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240843,0.000000,0.095861,...,0.000000,0.000000,0.000000,0.000000,0.0,0.108082,0.117763,0.000000,0.000000,0.000000
3,0.008403,0.000000,1.000000,0.179391,0.011294,0.000000,0.072246,0.000000,0.000000,0.000000,...,0.006560,0.000000,0.068686,0.000000,0.0,0.020322,0.022142,0.000000,0.000000,0.089849
4,0.040571,0.000000,0.179391,1.000000,0.054530,0.000000,0.348828,0.000000,0.000000,0.000000,...,0.031674,0.101979,0.567487,0.000000,0.0,0.098119,0.106908,0.365843,0.000000,0.433821
5,0.011755,0.000000,0.011294,0.054530,1.000000,0.000000,0.640342,0.000000,0.000000,0.000000,...,0.009177,0.000000,0.096091,0.000000,0.0,0.028429,0.030976,0.000000,0.000000,0.125697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.348295,0.108082,0.020322,0.098119,0.028429,0.211466,0.039515,0.000000,0.397065,0.213809,...,0.206804,0.631077,0.172901,0.657123,0.0,1.000000,0.917791,0.000000,0.767770,0.226174
193583,0.379492,0.117763,0.022142,0.106908,0.030976,0.000000,0.043055,0.000000,0.000000,0.000000,...,0.169874,0.687605,0.188388,0.715984,0.0,0.917791,1.000000,0.000000,0.612800,0.246433
193585,0.000000,0.000000,0.000000,0.365843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.278750,0.644671,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000
193587,0.232553,0.000000,0.000000,0.000000,0.000000,0.275428,0.000000,0.000000,0.517166,0.278480,...,0.247849,0.821961,0.000000,0.855885,0.0,0.767770,0.612800,0.000000,1.000000,0.000000


In [22]:
print(cs_df[1].sort_values(ascending=False))

2         1.000000
46972     0.322201
158813    0.300850
119655    0.300850
80748     0.300850
            ...   
4921      0.000000
4920      0.000000
4919      0.000000
4917      0.000000
193609    0.000000
Name: 1, Length: 9742, dtype: float64


In [23]:
print(movies_df.loc[1])
print(movies_df.loc[46972])
print(movies_df.loc[126142])
print(movies_df.loc[2043])
print(movies_df.loc[2399])

title                                Toy Story (1995)
genres    Adventure|Animation|Children|Comedy|Fantasy
Name: 1, dtype: object
title     Night at the Museum (2006)
genres    Action|Comedy|Fantasy|IMAX
Name: 46972, dtype: object
title     The Cave of the Golden Rose (1991)
genres            Adventure|Children|Fantasy
Name: 126142, dtype: object
title     Darby O'Gill and the Little People (1959)
genres                   Adventure|Children|Fantasy
Name: 2043, dtype: object
title     Santa Claus: The Movie (1985)
genres       Adventure|Children|Fantasy
Name: 2399, dtype: object


## ContentBasedProfile(Just Test)

In [24]:
user_one = ratings_df[ratings_df['userId']==1]
user_one = user_one.set_index(user_one.movieId).drop(['userId','movieId','timestamp'], 1)
user_one

,rating
movieId,
1,4.0
3,4.0
6,4.0
47,5.0
50,5.0
...,...
3744,4.0
3793,5.0
3809,4.0


In [25]:
user_one_rp = movie_representation[movie_representation.index.isin(user_one.index)]
user_one_rp

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,...,women,wonderwoman,workplace,writing,wrongful imprisonment,wry,younger men,zither,zoe kazan,zombies
1,0.0,0.000000,0.887245,1.202607,1.16648,0.413923,0.000000,0.0,0.000000,1.097111,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.00000,0.413923,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.726672,0.000000,0.000000,0.00000,0.000000,0.909829,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.909829,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3744,0.0,0.726672,0.000000,0.000000,0.00000,0.000000,0.909829,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3793,0.0,0.726672,0.887245,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3809,0.0,0.000000,0.000000,0.000000,0.00000,0.413923,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4006,0.0,0.000000,0.887245,1.202607,1.16648,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
user_one_profile = np.dot(user_one_rp.T, user_one)

In [27]:
user_one_rc = movie_representation.dot(user_one_profile) / user_one_profile.sum()
user_one_rc.columns = ['score']

In [28]:
user_one_rc.score.sort_values(ascending=False)

296       0.801246
2959      0.370867
1732      0.247237
260       0.210034
79132     0.181950
            ...   
94130     0.000000
94735     0.000000
7225      0.000000
154358    0.000000
155589    0.000000
Name: score, Length: 9742, dtype: float64

In [29]:
ratings_df[ratings_df['userId']==1][ratings_df['movieId']==296]

<ipython-input-29-6a4a6c0e3d4e>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ratings_df[ratings_df['userId']==1][ratings_df['movieId']==296]


,userId,movieId,rating,timestamp
16,1,296,3.0,964982967


## Evaluation

In [30]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=42)
test_userids = list(set(test_df.userId.values))

print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


In [33]:
%%time
result_df = pd.DataFrame()

for user_id in test_userids:
    user_record_df = train_df.loc[train_df.userId == int(user_id), :] 
    
    user_sim_df = cs_df.loc[user_record_df['movieId']] 
    user_rating_df = user_record_df[['rating']]  # (n, 1)
    sim_sum = np.sum(user_sim_df.T.to_numpy(), -1)  # (9742, 1)
    
    prediction = np.matmul(user_sim_df.T.to_numpy(), user_rating_df.to_numpy()).flatten() / (sim_sum+1)
    
    prediction_df = pd.DataFrame(prediction, index=cs_df.index).reset_index()
    prediction_df.columns = ['movieId', 'pred_rating']    
    prediction_df = prediction_df[['movieId', 'pred_rating']][prediction_df.movieId.isin(test_df[test_df.userId == user_id]['movieId'].values)]

    temp_df = prediction_df.merge(test_df[test_df.userId == user_id], on='movieId')
    result_df = pd.concat([result_df, temp_df], axis=0)

Wall time: 8.78 s


In [34]:
result_df.head(10)

,movieId,pred_rating,userId,rating,timestamp
0,151,4.293969,1,5.0,964984041
1,423,4.082067,1,3.0,964982363
2,596,4.182728,1,5.0,964982838
3,673,4.147920,1,3.0,964981775
4,1029,4.264604,1,5.0,964982855
5,1198,3.768049,1,5.0,964981827
6,1214,3.603542,1,4.0,964981855
7,1278,3.858380,1,5.0,964983414
8,1282,4.182728,1,5.0,964982703
9,1445,4.291765,1,3.0,964984112
